In [1]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import pickle, threading
from keras.utils import to_categorical
import linecache

epochs = 50
trainable=True

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

sys.version_info[0]

3

In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [4]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [ ]:
def doc_load(filename):
    f = open(filename,'rb')
    text = f.read().decode()
    f.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    f = open(filename,'wb')
    f.write(data.encode())
    f.close()


In [ ]:
text = doc_load('/home/jindal/notebooks/wikipedia/de-wiki_tokenized.txt')

In [ ]:
print(text[:200])

In [5]:
window_size =51

In [ ]:
def generate_sequences(out_file, in_file ) -> None:
    x = 0
    output = open(out_file,'wb')
    with open(in_file,'rb') as f:
        for line in f:
            if x==1000: #To generate toy dataset
                break
            x+=1
            text = line.decode().split()
            for i in range(len(text)):
                temp = ['0' for i in range(window_size - (i +1))] + text[:i+1]
                string = ' '.join(temp)+' \n'
                output.write(string.encode())
    output.close()

In [ ]:
generate_sequences('toy_dataset_sequences','/home/jindal/notebooks/wikipedia/de-wiki_tokenized.txt')

In [6]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [ ]:
toy_file = open('toy_dataset_sequences','wb')

n_lines=0
with open('wiki_sequences_lm','rb') as f:
    for line in f:
        if n_lines== 100:
            break
        n_lines+=1
        toy_file.write(line)
#         toy_file.write('\n'.encode())
toy_file.close()

In [7]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/toy_dataset_sequences_shuffled') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [8]:
print(characters)

{'ί': True, 'Y': True, 'Z': True, '&': True, 'x': True, '∪': True, 'í': True, 'm': True, 'κ': True, ')': True, 'K': True, 'ô': True, 'C': True, 'v': True, '¬': True, '+': True, 'V': True, '’': True, 'z': True, 'o': True, 'E': True, 'q': True, 'O': True, 'U': True, 'A': True, '%': True, 'b': True, 'p': True, '?': True, '#': True, 'μ': True, 'F': True, 'G': True, '‘': True, '†': True, 'M': True, 'k': True, 'D': True, 'a': True, 'L': True, 'T': True, 'X': True, 'Δ': True, '∧': True, '*': True, 'H': True, '€': True, 'é': True, 'Ö': True, 'Q': True, 'W': True, 'ß': True, 'i': True, '–': True, 'S': True, '…': True, 'Ä': True, 'J': True, '-': True, 'N': True, 'c': True, 't': True, '∨': True, '·': True, 'Ω': True, '_': True, 'l': True, 'g': True, 'e': True, '↔': True, ',': True, 'r': True, 'h': True, 'ö': True, 'ä': True, 'τ': True, '=': True, '!': True, 'y': True, 'ἀ': True, '"': True, '„': True, '0': True, '‚': True, '“': True, '°': True, '~': True, 's': True, 'B': True, 'j': True, 'n': True

In [9]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [10]:
toy_vocab ={}

with open('toy_dataset_sequences') as f:
    for line in f:
#         print(line)
        line = line.split()
#         print(len(line))
        for word in line:
#             word = word
#             print(word)
            toy_vocab[word]=True

In [11]:
print(toy_vocab)

{'gewinnen': True, 'Puccini': True, 'jüngster': True, 'erlangten': True, 'Lesart': True, 'bekanntesten': True, 'Wohnungen': True, 'Drehbuchwettbewerbs': True, 'entgegenzusetzen': True, 'Stoffmenge': True, 'Mrs': True, 'Außerhalb': True, 'Zahlung': True, 'günstiger': True, 'Plains': True, 'Bau': True, 'drängte': True, 'Regisseur': True, 'hinreichender': True, 'dem': True, 'Filmeinfuhrverbot': True, 'Kuhn': True, 'ältester': True, 'lebenstauglicheres': True, 'früheren': True, 'bestehenden': True, 'lokale': True, 'Verlobte': True, 'innerhalb': True, 'konvertierte': True, 'Presents': True, 'abzuschließen': True, 'Beta': True, 'kontinuierliche': True, 'Schauspiel': True, 'mehrwertigen': True, 'Fassung': True, 'äquivalent': True, 'arbeiten': True, 'Schule': True, 'seither': True, 'Villen': True, 'sei': True, 'Oxiden': True, 'aufstellt': True, 'spät': True, 'Silbermünzen': True, 'kritisieren': True, 'Fünfjahresverträge': True, 'Satzes': True, 'algebraischer': True, 'this': True, 'atomaren': T

In [12]:
line = linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/toy_dataset_sequences', 17)

print(line)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 Alan Smithee steht als Pseudonym für einen fiktiven Regisseur , der Filme verantwortet , bei 



In [13]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [14]:
char2Idx['0']


82

In [15]:
print(word2Idx)

{'Chlotrudis': 4676, 'gewinnen': 0, 'Puccini': 1, 'jüngster': 2, 'ihr': 812, 'Lesart': 4, 'bekanntesten': 5, 'Wohnungen': 6, 'Drehbuchwettbewerbs': 7, 'entgegenzusetzen': 8, 'mütterlicherseits': 4696, 'Stoffmenge': 9, 'Mrs': 10, 'Außerhalb': 11, 'Zahlung': 12, 'günstiger': 13, 'Plains': 14, 'Bau': 15, 'Regisseur': 17, 'hinreichender': 18, 'dem': 19, 'Filmeinfuhrverbot': 20, 'ältester': 22, 'lebenstauglicheres': 23, 'früheren': 24, 'bestehenden': 25, 'lokale': 26, 'Verlobte': 27, 'innerhalb': 28, 'konvertierte': 29, 'neben': 2416, 'führten': 3312, 'Goldene': 4043, 'Beta': 32, 'kontinuierliche': 33, 'Schauspiel': 34, 'mehrwertigen': 35, 'Fassung': 36, 'äquivalent': 37, 'arbeiten': 38, 'entwickelte': 2418, 'seither': 40, 'Villen': 41, 'sei': 42, 'Oxiden': 43, 'aufstellt': 44, 'spät': 45, 'Silbermünzen': 46, 'kritisieren': 47, 'stille': 3479, 'Fünfjahresverträge': 48, 'Satzes': 49, 'algebraischer': 50, 'this': 51, 'atomaren': 52, 'missbrauchen': 1598, 'Rahmen': 54, 'Totten': 55, 'qualitati

In [16]:
line_number=0
# lock = threading.Lock()

In [41]:
def my_generator(file :"input training file", batch_size):
    
    global line_number, lock

    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
#     batch_features = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims)) # initializing features with zeros
        output_labels = []
        # print(len(features))
        for i in range(batch_size):
            
#             lock.acquire()
            index = line_number%20869 +1
#             print(index)
            line_number+=1
#             lock.release()
#             index = random.choice(len(features), 1)[0]
#             print(index)
            line = linecache.getline(file, index)
            line = line.split()
#             print(line)
            temp_casing = []
            temp_char=[]
            temp_word=[]
#             print(line)
            if len(line)!=51:
#                 print(index)
                continue
            for word in line[:-1]:
#                 print(word)
                casing =getCasing(word, case2Idx)
#                 print(casing)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
#                 print(temp_char2)
#                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
#                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
#             print(temp_word)
#             print(len(temp_word))
#             print(temp_casing)
#             print(temp_char)
#             print(len(temp_char))
#             print(" **************** ")
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(word2Idx[line[-1]])
#             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
#             batch_labels[i] = labels[index]
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

In [43]:
for inp, output in my_generator('toy_dataset_sequences_shuffled',10):
    print(inp[0].shape) # word
    print(inp[1].shape) # casing
    print(inp[2].shape) # char
    print(output.shape)

(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(9, 50, 300)
(9, 50)
(9, 50, 52)
(9,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(8, 50, 300)
(8, 50)
(8, 50, 52)
(8,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)


(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)
(9, 50, 300)
(9, 50)
(9, 50, 52)
(9,)
(10, 50, 300)
(10, 50)
(10, 50, 52)
(10,)


KeyboardInterrupt: 

In [18]:
vocab_size = len(word2Idx.keys())+1
print(vocab_size)

4851


In [19]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [44]:
words_input = Input(shape=(None, 300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3300        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [21]:

20700//32

646

In [38]:
line_number=0
model.fit_generator(my_generator('toy_dataset_sequences_shuffled', 32), epochs=1, steps_per_epoch=20869//32)

Epoch 1/1
16
168
199
  1/652 [..............................] - ETA: 3:53 - loss: 4.7048 - acc: 0.1290316
355
365
  4/652 [..............................] - ETA: 3:05 - loss: 5.0994 - acc: 0.1026462
471
476
 18/652 [..............................] - ETA: 2:38 - loss: 5.1733 - acc: 0.1259912
924
 19/652 [..............................] - ETA: 2:37 - loss: 5.1715 - acc: 0.1209938
946
959
 27/652 [>.............................] - ETA: 2:31 - loss: 5.0964 - acc: 0.11781200
1214
 53/652 [=>............................] - ETA: 2:22 - loss: 4.9594 - acc: 0.12422023
2046
 62/652 [=>............................] - ETA: 2:20 - loss: 4.8836 - acc: 0.13312310
2321
2331
 76/652 [==>...........................] - ETA: 2:18 - loss: 4.8485 - acc: 0.12662760
2774
 79/652 [==>...........................] - ETA: 2:17 - loss: 4.8297 - acc: 0.12982861
2868
107/652 [===>..........................] - ETA: 2:10 - loss: 4.8100 - acc: 0.12483746
3769
111/652 [====>.........................] - ETA: 2:09 - loss:

226/652 [=========>....................] - ETA: 1:40 - loss: 4.7759 - acc: 0.12497560
7565
231/652 [=========>....................] - ETA: 1:39 - loss: 4.7784 - acc: 0.12537725
7741
243/652 [==========>...................] - ETA: 1:36 - loss: 4.7755 - acc: 0.12648111
8124
274/652 [===========>..................] - ETA: 1:29 - loss: 4.7795 - acc: 0.12889097
9112
9116
286/652 [============>.................] - ETA: 1:26 - loss: 4.7769 - acc: 0.12969497
9500
291/652 [============>.................] - ETA: 1:25 - loss: 4.7754 - acc: 0.13009637
9645
294/652 [============>.................] - ETA: 1:24 - loss: 4.7755 - acc: 0.13039731
9732
9744
297/652 [============>.................] - ETA: 1:23 - loss: 4.7783 - acc: 0.13039840
9841
9844
313/652 [=============>................] - ETA: 1:19 - loss: 4.7897 - acc: 0.128910343
10363
10365
317/652 [=============>................] - ETA: 1:18 - loss: 4.7850 - acc: 0.130210465
10485
325/652 [=============>................] - ETA: 1:16 - loss: 4.78

437/652 [===================>..........] - ETA: 50s - loss: 4.8086 - acc: 0.130514321
14324
444/652 [===================>..........] - ETA: 48s - loss: 4.8098 - acc: 0.130814530
14554
452/652 [===================>..........] - ETA: 47s - loss: 4.8152 - acc: 0.130714791
14809
456/652 [===================>..........] - ETA: 46s - loss: 4.8169 - acc: 0.130614926
14938
461/652 [====================>.........] - ETA: 44s - loss: 4.8212 - acc: 0.130815073
15082
15098
465/652 [====================>.........] - ETA: 43s - loss: 4.8284 - acc: 0.130915215
15229
471/652 [====================>.........] - ETA: 42s - loss: 4.8394 - acc: 0.130915405
15408
486/652 [=====================>........] - ETA: 38s - loss: 4.8641 - acc: 0.129715879


ResourceExhaustedError: OOM when allocating tensor with shape[400,4851] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Nadam/mul_73 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Nadam/sub_31, training/Nadam/gradients/dense_1/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Nadam/mul_73', defined at:
  File "/home/jindal/miniconda3/envs/NER2/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jindal/miniconda3/envs/NER2/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-pa

In [39]:
print(vocab_size)

4851


In [ ]:
y = np.array(y)
word_inp = np.array(word_inp)
casing_inp = np.array(casing_inp)
char_inp = np.array(char_inp)

In [ ]:
print(word_inp.shape)
print(casing_inp.shape)
print(char_inp.shape)

In [ ]:
model.fit([word_inp,casing_inp, char_inp],y,epochs=1)

In [ ]:
model.save('german_ner_lm.h5')